Import packages

In [168]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

Data Understunding

In [169]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Read CSV into DataFrame") \
    .getOrCreate()

# Read CSV file into DataFrame
df = spark.read.load('data/googleplaystore.csv', format='csv', sep=',', header=True, inferSchema=True)

In [170]:
df.count()

10841

In [171]:
df.show()

+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|                 App|      Category|Rating|Reviews|Size|   Installs|Type|Price|Content Rating|              Genres|      Last Updated|       Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+-----------+----+-----+--------------+--------------------+------------------+------------------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M|    10,000+|Free|    0|      Everyone|        Art & Design|   January 7, 2018|             1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967| 14M|   500,000+|Free|    0|      Everyone|Art & Design;Pret...|  January 15, 2018|             2.0.0|4.0.3 and up|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510|8.7M| 5,000,000+|Free|    0|      Everyone|        Art & Design|    August 1, 2018|             1.2.4|4.0.3 

In [172]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning

In [173]:
df = df.drop('Size', 'Content Rating', 'Last Updated', 'Android Ver', 'Current Ver')

In [174]:
df.columns

['App', 'Category', 'Rating', 'Reviews', 'Installs', 'Type', 'Price', 'Genres']

In [175]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [176]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType, DoubleType

df = df.withColumn('Rating', col('Rating').cast(DoubleType())) \
       .withColumn('Reviews', col('Reviews').cast(IntegerType())) \
        .withColumn('Installs', regexp_replace('Installs', '[^0-9]', '')) \
        .withColumn('Installs', col('Installs').cast(IntegerType())) \
        .withColumn('Price', regexp_replace('Price', '\$', '')) \
       .withColumn('Price', col('Price').cast(DoubleType()))

In [177]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Genres: string (nullable = true)



In [178]:
from pyspark.sql.functions import isnan, col, sum

nan_count = df.select(sum(isnan(col("Rating")).cast("int")).alias("NaN_Count")).collect()[0]["NaN_Count"]

print(nan_count)

1474


In [179]:
# Remplacer les valeurs NaN par 0.0 dans la colonne "Rating"
df = df.fillna(0.0, subset=["Rating"])

In [180]:
from pyspark.sql.functions import isnan, col, sum

nan_count = df.select(sum(isnan(col("Rating")).cast("int")).alias("NaN_Count")).collect()[0]["NaN_Count"]

print(nan_count)

0


In [181]:
df.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|  0.0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|  0.0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

In [182]:
df.createOrReplaceTempView('apps')

In [183]:
# Exécuter la requête SQL
result = spark.sql("SELECT * FROM apps")

# Afficher le résultat
result.show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|  0.0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|  0.0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|  0.0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|  0.0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|  0.0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|  0.0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|  0.0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

Top 10 Reviews given to apps

In [184]:
# Exécuter la requête SQL
top_10 = spark.sql("SELECT App, SUM(Reviews) AS TotalReviews FROM apps GROUP BY App ORDER BY TotalReviews DESC LIMIT 10")

# Afficher le résultat
top_10.show()

+--------------------+------------+
|                 App|TotalReviews|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+



Top 10 installs Free apps

In [185]:
# Exécuter la requête SQL
topfree_10 = spark.sql("SELECT App As FreeApps, SUM(Installs) AS TotalInstalls FROM apps Where Type='Free' GROUP BY App ORDER BY TotalInstalls DESC LIMIT 10")

# Afficher le résultat
topfree_10.show()

+--------------------+-------------+
|            FreeApps|TotalInstalls|
+--------------------+-------------+
|      Subway Surfers|   6000000000|
|           Instagram|   4000000000|
|            Hangouts|   4000000000|
|        Google Drive|   4000000000|
|         Google News|   4000000000|
|       Google Photos|   4000000000|
|    Candy Crush Saga|   3500000000|
|  WhatsApp Messenger|   3000000000|
|Messenger – Text ...|   3000000000|
|Google Chrome: Fa...|   3000000000|
+--------------------+-------------+



Top 10 installs Paid apps

In [186]:
# Exécuter la requête SQL
toppaid_10 = spark.sql("SELECT App As PaidApps, SUM(Installs) AS TotalInstalls FROM apps Where Type='Paid' GROUP BY App ORDER BY TotalInstalls DESC LIMIT 10")

# Afficher le résultat
toppaid_10.show()

+--------------------+-------------+
|            PaidApps|TotalInstalls|
+--------------------+-------------+
|           Minecraft|     20000000|
|       Hitman Sniper|     10000000|
| Facetune - For Free|      3000000|
|Beautiful Widgets...|      2000000|
|          HD Widgets|      2000000|
|              Tasker|      1000000|
| Fruit Ninja Classic|      1000000|
|          True Skate|      1000000|
|   Where's My Water?|      1000000|
| DraStic DS Emulator|      1000000|
+--------------------+-------------+



Top paid apps

In [187]:
# Exécuter la requête SQL
toppaid = spark.sql("SELECT App , SUM(Price) AS TotalPrice FROM apps Where Type='Paid' GROUP BY App ORDER BY TotalPrice DESC LIMIT 10")

# Afficher le résultat
toppaid.show()

+--------------------+----------+
|                 App|TotalPrice|
+--------------------+----------+
|I'm Rich - Trump ...|     400.0|
|most expensive ap...|    399.99|
|  I am rich(premium)|    399.99|
|   I Am Rich Premium|    399.99|
|      I am Rich Plus|    399.99|
|I'm Rich/Eu sou R...|    399.99|
|       I Am Rich Pro|    399.99|
|  I AM RICH PRO PLUS|    399.99|
|           I am Rich|    399.99|
|          I am Rich!|    399.99|
+--------------------+----------+



Top paid rating apps

In [190]:
# Exécuter la requête SQL
toppaid_rating = spark.sql("SELECT App , Rating  FROM apps Where Type='Paid' ORDER BY Rating DESC LIMIT 10")

# Afficher le résultat
toppaid_rating.show()

+--------------------+------+
|                 App|Rating|
+--------------------+------+
|     P-Home for KLWP|   5.0|
|            Ra Ga Ba|   5.0|
|Android P Style I...|   5.0|
|Super Hearing Sec...|   5.0|
|Easy Hotspot Ad Free|   5.0|
|AJ Gray Dark Icon...|   5.0|
|AP Art History Fl...|   5.0|
|   AJ Blue Icon Pack|   5.0|
|        ADS-B Driver|   5.0|
|meStudying: AP En...|   5.0|
+--------------------+------+

